In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp MLPipeline

# Training Pipeline

> An API to create super fast training pipeline for machine learning models based on tabular or strucuture data

> It comes with model parallelism and cutting edge hyperparameter tuning techniques.

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev import *

In [ ]:
# export
from tabular_ml_toolkit.DataFrameLoader import *
from tabular_ml_toolkit.PreProcessor import *

In [ ]:
# export
# hide
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, roc_auc_score,accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from tune_sklearn import TuneGridSearchCV


# for displaying diagram of pipelines 
from sklearn import set_config
set_config(display="diagram")

# for finding n_jobs in all sklearn estimators
from sklearn.utils import all_estimators
import inspect

# Just to compare fit times
import time

# for os specific settings
import os

In [ ]:
# export

#Global variable

IDEAL_CPU_CORES = os.cpu_count()-1

class MLPipeline:
    """
    Represent MLPipeline class
    
    Attributes:\n
    pipeline: An MLPipeline instance \n
    dfl: A DataFrameLoader instance \n
    pp: A PreProcessor Instance \n
    model: The given Model
    """

    def __init__(self):
        self.dfl = None
        self.pp = None
        self.model = None
        self.spl = None
        self.transformer_type = None
        self.has_n_jobs = ['XGBRegressor', 'XGBClassifier']
        self.create_has_n_jobs()
    
    def __str__(self):
        """Returns human readable string reprsentation"""
        attr_str = ("spl, dfl, pp, model")
        return ("Training Pipeline object with attributes:"+attr_str)
    
    def __repr__(self):
        return self.__str__()

    
    # core methods
    def create_has_n_jobs(self):
        for est in all_estimators():
            s = inspect.signature(est[1])
            if 'n_jobs' in s.parameters:
                self.has_n_jobs.append(est[0])
    
    # Bundle preprocessing and modeling code in a training pipeline
    def bundle_preproessor_model(self, transformer_type, model):
        self.spl = Pipeline(
            steps=[('preprocessor', transformer_type),
                   ('model', model)])
    
    # Core methods for Simple Training
    def prepare_data_for_training(self, train_file_path:str,
                                  test_file_path:str,
                                  idx_col:str, target:str,
                                  random_state:int,
                                  model:object):
        
        # check if given model supports n_jobs aka cpu core based Parallelism
        estimator_name = model.__class__.__name__
        # print(estimator_name)
        # print(self.has_n_jobs)
        if estimator_name in self.has_n_jobs :
            # In order to OS not to kill the job, leave one processor out 
            model.n_jobs = IDEAL_CPU_CORES
            self.model = model
        else:
            print(f"{estimator_name} doesn't support parallelism yet! Training will continue on a single thread.")
            self.model = model
        
        # call DataFrameLoader module
        self.dfl = DataFrameLoader().from_csv(
            train_file_path=train_file_path,
            test_file_path=test_file_path,
            idx_col=idx_col,
            target=target,
            random_state=random_state)
        
        # call PreProcessor module
        self.pp = PreProcessor().preprocess_all_cols(dataframeloader=self.dfl)
        
        # call bundle method
        self.bundle_preproessor_model(transformer_type=self.pp.transformer_type,
                                     model = model)
        return self
    
    # Force to update the preprocessor in pipeline
    def update_preprocessor(self, num_imputer = SimpleImputer(strategy='median'),
                            num_scaler = StandardScaler(),
                            cat_imputer = SimpleImputer(strategy='constant')):
        # change preprocessor
        self.pp = PreProcessor().preprocess_all_cols(self.dfl,
                                                     num_imputer=num_imputer,
                                                     num_scaler=num_scaler,
                                                     cat_imputer=cat_imputer)
        # recall bundle method
        self.bundle_preproessor_model(transformer_type=self.pp.transformer_type,
                                     model = self.model)
        
    
    # Force to update the model in pipeline
    def update_model(self, model:object):
        #change model
        self.model = model
        # recall bundle method
        self.bundle_preproessor_model(transformer_type=self.pp.transformer_type,
                                     model = model)
    
    
    def do_cross_validation(self, cv:int, scoring:str):
        scores = cross_val_score(
            estimator=self.spl,
            X=self.dfl.X,
            y=self.dfl.y,
            scoring=scoring,
            cv=cv)
        # Multiply by -1 since sklearn calculates *negative* scoring for some of the metrics
        if "neg_" in scoring:
            scores = -1 * scores
        return scores
        
    # Core methods for GridSearch
    def do_grid_search(self, param_grid:object, cv:int,
                       scoring:str, n_jobs:int=IDEAL_CPU_CORES):
        
        # create GridSeachCV instance
        grid_search = GridSearchCV(estimator=self.spl,
                                   param_grid=param_grid,
                                   cv=cv,
                                   scoring=scoring,
                                   n_jobs=n_jobs)
        # now call fit
        grid_search.fit(self.dfl.X, self.dfl.y)
        return grid_search
    
    # Core methods for Tune SK-Learn GridSearch
    def do_tune_grid_search(self,
                            param_grid:object,
                            scoring:str=None,
                            mode:str='max',
                            cv:int=5,
                            early_stopping=True,
                            time_budget_s:int=None,
                            name:str=None,
                            use_gpu:bool=False,
                            stopper:object=None,
                            max_iters:int=10,
                            n_jobs:int=IDEAL_CPU_CORES):
        
        # create GridSeachCV instance
        tune_search = TuneGridSearchCV(
            estimator=self.spl,
            param_grid=param_grid,
            scoring=scoring,
            mode=mode,
            cv=cv,
            time_budget_s=time_budget_s,
            name=name,
            use_gpu=use_gpu,
            early_stopping=early_stopping,
            stopper=stopper,
            max_iters=max_iters,
            n_jobs=n_jobs,
            pipeline_auto_early_stop=True)
        
        # now call fit
        tune_search.fit(self.dfl.X, self.dfl.y)
        return tune_search

    
    # do k-fold training
    def do_k_fold_training(self, n_splits:int, metrics:object, random_state=42):
        
        #create stratified K Folds instance
        k_fold = StratifiedKFold(n_splits=n_splits,
                             random_state=random_state,
                             shuffle=True)
        
        # list contains metrics score for each fold
        metrics_score = []
        n=0
        for train_idx, valid_idx in k_fold.split(self.dfl.X, self.dfl.y):
            # create X_train
            self.dfl.X_train = self.dfl.X.iloc[train_idx]
            # create X_valid
            self.dfl.X_valid = self.dfl.X.iloc[valid_idx] 
            # create y_train
            self.dfl.y_train = self.dfl.y.iloc[train_idx]
            # create y_valid
            self.dfl.y_valid = self.dfl.y.iloc[valid_idx]
            
            # fit
            self.spl.fit(self.dfl.X_train, self.dfl.y_train)
            
            #evaluate metrics based upon input
            if "proba" in metrics.__globals__:
                metrics_score.append(metrics(self.dfl.y_valid,
                                               self.spl.predict_proba(self.dfl.X_valid)[:,1]))
            else:
                metrics_score.append(metrics(self.dfl.y_valid,
                                               self.spl.predict(self.dfl.X_valid)))
            
            print(f"fold: {n+1} , {str(metrics.__name__)}: {metrics_score[n]}")
            # increment fold counter label
            n += 1
        return k_fold, metrics_score
            
    def do_k_fold_prediction(self, k_fold:object):
        # create preds dataframe
        preds = np.zeros(self.dfl.X_test.shape[0])
        for _ in range(k_fold.n_splits):
            # predict
            preds += self.spl.predict(self.dfl.X_test) / k_fold.n_splits
        return preds

In [ ]:
show_doc(MLPipeline)

<h2 id="MLPipeline" class="doc_header"><code>class</code> <code>MLPipeline</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>MLPipeline</code>()

Represent MLPipeline class

Attributes:

pipeline: An MLPipeline instance 

dfl: A DataFrameLoader instance 

pp: A PreProcessor Instance 

model: The given Model

#### Build MLPipeline Class with House Data

*You can use MLPipeline to train any model. Here we are using RandomForestRegressor from Scikit-Learn, on  [Melbourne Home Sale price data](https://www.kaggle.com/estrotococo/home-data-for-ml-course)*

In [ ]:
#export
# Dataset file names and Paths
DIRECTORY_PATH = "https://raw.githubusercontent.com/psmathur/tabular_ml_toolkit/master/input/home_data/"
TRAIN_FILE = "train.csv"
TEST_FILE = "test.csv"
SAMPLE_SUB_FILE = "sample_submission.csv"

In [ ]:
#export
# create scikit-learn ml model
scikit_model = RandomForestRegressor(n_estimators=200, random_state=42)

# createm ml pipeline for scikit-learn model
tmlt = MLPipeline().prepare_data_for_training(
    train_file_path= DIRECTORY_PATH+TRAIN_FILE,
    test_file_path= DIRECTORY_PATH+TEST_FILE,
    idx_col="Id", target="SalePrice",
    model=scikit_model,
    random_state=42)

In [ ]:
show_doc(tmlt.prepare_data_for_training)

<h4 id="MLPipeline.prepare_data_for_training" class="doc_header"><code>MLPipeline.prepare_data_for_training</code><a href="__main__.py#L50" class="source_link" style="float:right">[source]</a></h4>

> <code>MLPipeline.prepare_data_for_training</code>(**`train_file_path`**:`str`, **`test_file_path`**:`str`, **`idx_col`**:`str`, **`target`**:`str`, **`random_state`**:`int`, **`model`**:`object`)



In [ ]:
tmlt.spl

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'Lo...
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', 'KitchenQual',
                                                   'Functional', 'FireplaceQu', ...]),
                                                 ('high_card_cat_cols',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Neighborhood',
                                                   'Exterior1st',
                                                   'Exterior2nd'])])),
                ('model',
                 RandomForestRegressor(n_estimators=200, n_jobs=11,
                                       random_state=42))])

In [ ]:
tmlt.dfl.create_train_valid(valid_size=0.2)

In [ ]:
show_doc(tmlt.dfl.create_train_valid)

<h4 id="DataFrameLoader.create_train_valid" class="doc_header"><code>DataFrameLoader.create_train_valid</code><a href="https://github.com/psmathur/tabular_ml_toolkit/tree/master/tabular_ml_toolkit/DataFrameLoader.py#L109" class="source_link" style="float:right">[source]</a></h4>

> <code>DataFrameLoader.create_train_valid</code>(**`valid_size`**:`float`, **`random_state`**=*`42`*)



In [ ]:
start = time.time()
# Now fit
tmlt.spl.fit(tmlt.dfl.X_train, tmlt.dfl.y_train)
end = time.time()
print("Fit Time:", end - start)

#predict
preds = tmlt.spl.predict(tmlt.dfl.X_valid)
print('X_valid MAE:', mean_absolute_error(tmlt.dfl.y_valid, preds))

Fit Time: 1.9442250728607178
X_valid MAE: 17483.238390410956


In [ ]:
show_doc(tmlt.spl.fit)

<h4 id="Pipeline.fit" class="doc_header"><code>Pipeline.fit</code><a href="sklearn/pipeline.py#L363" class="source_link" style="float:right">[source]</a></h4>

> <code>Pipeline.fit</code>(**`X`**, **`y`**=*`None`*, **\*\*`fit_params`**)

Fit the model.

Fit all the transformers one after the other and transform the
data. Finally, fit the transformed data using the final estimator.

Parameters
----------
X : iterable
    Training data. Must fulfill input requirements of first step of the
    pipeline.

y : iterable, default=None
    Training targets. Must fulfill label requirements for all steps of
    the pipeline.

**fit_params : dict of string -> object
    Parameters passed to the ``fit`` method of each step, where
    each parameter name is prefixed such that parameter ``p`` for step
    ``s`` has key ``s__p``.

Returns
-------
self : object
    Pipeline with fitted steps.

In [ ]:
show_doc(tmlt.spl.fit)

<h4 id="Pipeline.fit" class="doc_header"><code>Pipeline.fit</code><a href="sklearn/pipeline.py#L363" class="source_link" style="float:right">[source]</a></h4>

> <code>Pipeline.fit</code>(**`X`**, **`y`**=*`None`*, **\*\*`fit_params`**)

Fit the model.

Fit all the transformers one after the other and transform the
data. Finally, fit the transformed data using the final estimator.

Parameters
----------
X : iterable
    Training data. Must fulfill input requirements of first step of the
    pipeline.

y : iterable, default=None
    Training targets. Must fulfill label requirements for all steps of
    the pipeline.

**fit_params : dict of string -> object
    Parameters passed to the ``fit`` method of each step, where
    each parameter name is prefixed such that parameter ``p`` for step
    ``s`` has key ``s__p``.

Returns
-------
self : object
    Pipeline with fitted steps.

#### let's update the numerical columns scaler to MinMaxScaler from StandardScaler and see if score improves

In [ ]:
tmlt.update_preprocessor(num_scaler=MinMaxScaler())

In [ ]:
show_doc(tmlt.update_preprocessor)

<h4 id="MLPipeline.update_preprocessor" class="doc_header"><code>MLPipeline.update_preprocessor</code><a href="__main__.py#L85" class="source_link" style="float:right">[source]</a></h4>

> <code>MLPipeline.update_preprocessor</code>(**`num_imputer`**=*`SimpleImputer(strategy='median')`*, **`num_scaler`**=*`StandardScaler()`*, **`cat_imputer`**=*`SimpleImputer(strategy='constant')`*)



In [ ]:
# check the udpated pipeline
tmlt.spl

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQ...
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', 'KitchenQual',
                                                   'Functional', 'FireplaceQu', ...]),
                                                 ('high_card_cat_cols',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Neighborhood',
                                                   'Exterior1st',
                                                   'Exterior2nd'])])),
                ('model',
                 RandomForestRegressor(n_estimators=200, n_jobs=11,
                                       random_state=42))])

In [ ]:
start = time.time()
# Now fit
tmlt.spl.fit(tmlt.dfl.X_train, tmlt.dfl.y_train)
end = time.time()
print("Fit Time:", end - start)

#predict
preds = tmlt.spl.predict(tmlt.dfl.X_valid)
print('X_valid MAE:', mean_absolute_error(tmlt.dfl.y_valid, preds))

Fit Time: 1.8192439079284668
X_valid MAE: 17464.29029109589


Indeed, the score did update, Depending upon your need, you can update numerical or categorical columns imputer too

In [ ]:
# hide
# run the script to build 

from nbdev.export import notebook2script; notebook2script()

Converted 00_DataFrameLoader.ipynb.
Converted 01_PreProcessor.ipynb.
Converted 02_MLPipeline.ipynb.
Converted 03_Tutorial.ipynb.
Converted 07_Kaggle_TPS_Tutorial.ipynb.
Converted index.ipynb.
